In [156]:
# 備考１(行番号)	備考２(顧客番号)	表示名	ﾌﾘｶﾞﾅ(顧客)	ﾌﾘｶﾞﾅ(会社)	会社名	ボトル1	ボトル1番号	ボトル2	ボトル2番号	ボトル3	ボトル3番号
# Remark 1 (line number) Remark 2 (customer number) Display name Furigana (customer)	Furigana (Company) Company name Bottle 1 Bottle 1 number Bottle 2 Bottle 2 number Bottle 3 Bottle 3 number

# 1	3	相田  得皓	ｱｲﾀﾞ　ﾄｸﾋﾛ	ｱｲﾀﾞｳﾝﾕ	相田運輸㈱
# 2	4	青木  道男	ｱｵｷ　ﾐﾁｵ	ｱｵｷﾓｸｻﾞｲ	青木木材㈱iiii
import pandas as pd
import numpy as np
import uuid
from datetime import datetime
import jaconv
import json

In [67]:
str(uuid.uuid4())

'9af1ce05-edca-4044-9b07-ebd9e148b601'

In [1]:
dfs = pd.read_excel("data.xlsx", sheet_name=None)

In [2]:
sheet = dfs["Sheet1"]
cols = sheet.columns

In [54]:
bots1 = sheet["ボトル1"][sheet["ボトル1"].notnull()]
bots2 = sheet["ボトル2"][sheet["ボトル2"].notnull()]
bots3 = sheet["ボトル3"][sheet["ボトル3"].notnull()]
bottles = set(bots1.to_list() + bots2.to_list() + bots3.to_list())

In [ ]:
  "drinks": {
    "7cfdc16a-817e-4ab2-ad89-1d3ac0fa5aaf": {
      "attributes": {
        "name": "Tequila",
        "url": "https://firebasestorage.googleapis.com/v0/b/barmanagerx.appspot.com/o/drinks%2F7cfdc16a-817e-4ab2-ad89-1d3ac0fa5aaf?alt=media&token=24c7e5c7-2163-4629-a414-e22443cb3ec3"
      },
      "createdAt": "2021-11-07T07:37:59.675Z",
      "updatedAt": "2021-11-07T07:37:59.675Z"
    },

In [190]:
datetime.utcnow().isoformat()[:-3]+ "Z"
# now_date = my_date.strftime("%Y-%m-%dT%H:%M:%S.000Z")


# "2022-04-22T05:23:26.080Z" = $1

'2022-04-22T05:24:37.905Z'

In [191]:
now_date = datetime.utcnow().isoformat()[:-3]+ "Z"



In [192]:
now_date

'2022-04-22T05:25:11.469Z'

In [193]:
# now_date = datetime.utcnow().isoformat()[:-3]+ "Z"

now_date2 = datetime.utcnow().isoformat()[:-3]+ "Z"



In [194]:
now_date2

'2022-04-22T05:25:17.904Z'

In [195]:
drinks = dict()
drinks_id = dict()
for bottle in bottles:
    uid = str(uuid.uuid4())
    drinks_id[bottle] = uid
    drinks[uid] = {
        "attributes": {
            "name": bottle,
            "url": ""
        },
        "createdAt": now_date,
        "updatedAt": now_date
    }
    

In [139]:
drinks_id['響'][]

{'響': {...},
 'バラン17': {...},
 'メーカーズ': {...},
 '碧': {...},
 'マッカランダブルカスク': {...},
 'マッカランファインオーク': {...},
 'クラウンロイヤル': {...},
 'ローゼス': {...},
 '竹鶴': {...},
 'グレンフィデック': {...},
 'ブラントン': {...},
 'トマーティン': {...},
 'ラフロイド': {...},
 'ハーパー': {...},
 'カナディアンクラブ': {...},
 'クルバジェ': {...},
 '黒響': {...},
 'グレングラント': {...},
 '竹鶴１７': {...},
 'ブレンダー響': {...},
 '竹鶴17': {...},
 'ヘネシー': {...},
 '白州１２': {...},
 '白州ノンビンテージ': {...},
 'ジャックダニエル': {...},
 '山崎': {...}}

In [ ]:
    "002a7479-f75f-463e-886b-e4fd2bc639f8": {
      "attributes": {
        "drinkCode": "23",
        "drinkId": "027bbc76-462a-49b3-adb1-dfe7b3366fde",
        "inUse": false,
        "userId": "8d12292d-4827-4904-9376-c455d614a184"
      },
      "createdAt": "2021-09-11T22:34:45.331931",
      "uid": "002a7479-f75f-463e-886b-e4fd2bc639f8",
      "updatedAt": "2021-09-21T22:34:45.331913"
    },

In [198]:
users = dict()
all_orders = dict()
for index, row in sheet.iterrows():
    na_row = row.isna()
#     my_bottles = []
    my_bottles = [(row[f"ボトル{i}"], str(row[f"ボトル{i}番号"])) for i in range(1,4) if not na_row[f"ボトル{i}"]]
    
    uid = str(uuid.uuid4())
    orders = dict()
    for my_bottle in my_bottles:
        uid_order = str(uuid.uuid4())
        
        orders[uid_order] = {
            "drinkCode": my_bottle[1],
            "drinkId": drinks_id[my_bottle[0]]
        }
        all_orders[uid_order] = {
          "attributes": {
            "drinkCode": my_bottle[1],
            "drinkId": drinks_id[my_bottle[0]],
            "inUse": False,
            "userId": uid
          },
          "createdAt": now_date,
          "uid": uid_order,
          "updatedAt": now_date
        }
    users[uid] = {
      "attributes": {
        "birthday": "",
        "email": "" if na_row['電子メール アドレス'] else row['電子メール アドレス'],
        "furigana": jaconv.h2z("" if na_row['ﾌﾘｶﾞﾅ(顧客)'] else row['ﾌﾘｶﾞﾅ(顧客)']),
        "introducer": "",
        "isInStore": False,
        "job": "" if na_row['会社名'] else  row['会社名'],
        "memo": "",
        "name": "" if na_row['表示名'] else row['表示名'],
        "phone": "" if na_row['携帯電話'] else row['携帯電話'],
        "photo": ""
      },
      "createdAt": now_date,
      "updatedAt": now_date2,
      "relationships": {
        "orders": orders
      }
    }
#     if(index > 200): break

In [199]:
all_data = dict()
all_data["bottles"] = drinks
all_data["users"] = users
all_data["orders"] = all_orders
with open('new_data.json', 'w', encoding='utf8') as fp:
    json.dump(all_data, fp, ensure_ascii=False)

In [159]:
with open('new_data.json', 'w', encoding='utf8') as fp:
    json.dump(all_data, fp, ensure_ascii=False)

SyntaxError: unexpected EOF while parsing (600063397.py, line 4)

float